In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import tensorflow as tf
from PIL import Image
import json
from pathlib import Path
from tensorflow import keras
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9

tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))


Using TensorFlow backend.


In [2]:
def preprocess(image):
    #resized_image = tf.image.resize(image,[224,224])
    final_image=keras.applications.resnet50.preprocess_input(image)
    return final_image


In [3]:
train_datagen = ImageDataGenerator(validation_split = 0.2,preprocessing_function=preprocess)

In [4]:
train_generator = train_datagen.flow_from_directory(
        r'D:\Downloads\Copy of shopee-product-detection-dataset\train_10\train_10',
        subset = 'training',# this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=16,
        color_mode="rgb",
        class_mode="categorical",
        shuffle=True,
        seed=42) 


val_generator = train_datagen.flow_from_directory(
        r'D:\Downloads\Copy of shopee-product-detection-dataset\train_10\train_10',
        subset = 'validation',# this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=16,
        color_mode="rgb",
        class_mode="categorical",
        shuffle=True,
        seed=42) 


Found 18575 images belonging to 10 classes.
Found 4638 images belonging to 10 classes.


In [5]:
pd.DataFrame(train_generator[0][1])

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
base_model=keras.applications.resnet50.ResNet50(weights="imagenet",include_top=False)

In [7]:
for layer in base_model.layers:
    layer.trainable=False

In [8]:
inputs=keras.Input(shape=(224,224,3))
base_model=base_model(inputs,training=False)

In [9]:
avg=keras.layers.GlobalAveragePooling2D()(base_model)
output=keras.layers.Dense(10,activation="softmax")(avg)
model=keras.Model(inputs=inputs,outputs=output)

In [10]:
optimizer=keras.optimizers.SGD(lr=0.2,momentum=0.9,decay=0.01)
model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])

In [ ]:
history=model.fit(train_generator,epochs=3,validation_data=val_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1161 steps, validate for 290 steps
Epoch 1/3
  65/1161 [>.............................] - ETA: 11:29 - loss: 30.1346 - accuracy: 0.7144

In [37]:
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)


model.save_weights("model_resnet.h5")

In [1]:
history

history


Epoch 1/5
   9/1161 [..............................] - ETA: 34:21 - loss: 25.4302 - accuracy: 0.3194

ResourceExhaustedError:  OOM when allocating tensor with shape[16,2048,7,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node gradients_3/activation_147/Relu_grad/ReluGrad-0-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_147558]

Function call stack:
keras_scratch_graph
